<a href="https://colab.research.google.com/github/IamJk28/aircraftgit/blob/main/Final_Training_Testing_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Setup Paths

In [ ]:
import os

In [ ]:
# this cell allows the google colab session to access your drive contents where we have uploaded images and Annotations
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Adjust var names appropriately - using the TF2 Object detection zoo @ https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md
# trying: faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8

CUSTOM_MODEL_NAME = 'Inception_ResNet50'
PRETRAINED_MODEL_NAME = 'faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8'#Change this for different model
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8.tar.gz'#Change for different model

TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [ ]:
folderpaths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME),
    'PROTOC_PATH':os.path.join('Tensorflow','protoc'),
    'TRAIN_PATH':os.path.join('Tensorflow','workspace','images','train'),
    'TEST_PATH':os.path.join('Tensorflow','workspace','images','test'),
    'COLLECT_PATH':os.path.join('Tensorflow','workspace','images','collectedimages')
 }

In [ ]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(folderpaths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME),
    'LABELMAP': os.path.join(folderpaths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
for path in folderpaths.values():
    if not os.path.exists(path):
      !mkdir -p {path}


#1. Untar Image and label Archive and move image and label folders to correct location

In [ ]:
#replace source_folder0 with link to tar.gz file of images and label folders

import shutil

source_folder0 = "drive/MyDrive/aircraftimage/final_dataset.tar.gz"
destination_folder0 = "Tensorflow/workspace/images"

if os.path.exists(source_folder0):
   shutil.copy(source_folder0, destination_folder0)

In [ ]:
#replace source_folder0 with the same tar.gz link/path as above

import tarfile

# Path to the archive file
source_folder0 = "/content/Tensorflow/workspace/images/final_dataset.tar.gz"

# Extract the contents of the archive
with tarfile.open(source_folder0, 'r:gz') as tar:
    tar.extractall("/content/Tensorflow/workspace/images")


In [ ]:
#use the first two source_folder strings in future. upload the images and Annotations files from Microsoft teams INTO a new file (here we have called it "aircraftimage" ,change the path of source folders if you call it something else)

#this code takes the "images" and "Annotations" files that you have uploaded into Google Drive and copies them in to the directory Tensorflow/workspace/collectedimages .from here using some later process/code we split the collected images(and xml files) into training and testing sets

#again make sure that the source_folder 1 and 2 paths are relevant to the tar.gz file that was just untarred.

source_folder1 = "Tensorflow/workspace/images/images"
source_folder2 = "Tensorflow/workspace/images/Annotations"
destination_folder = "Tensorflow/workspace/images/collectedimages"

source_folders = [source_folder1, source_folder2]

for source_folder in source_folders:
    source_files = os.listdir(source_folder)
    for file_name in source_files:
        source_file = os.path.join(source_folder, file_name)
        destination_file = os.path.join(destination_folder, file_name)
        shutil.copy(source_file, destination_file)

In [ ]:
#check the number of images you are running this experiment with

#define the directory you want to count images in
directory_path1 = "/content/Tensorflow/workspace/images/images"
directory_path2 = "/content/Tensorflow/workspace/images/Annotations"

#list all images in the directory
items = os.listdir(source_folder1)

#count
num_items = len(items)

#print the number of images you are working with
print(f"Number of items in the directory: {num_items}")

#list all images in the directory
items = os.listdir(source_folder2)

#count
num_items = len(items)

#print the number of images you are working with
print(f"Number of items in the directory: {num_items}")


Number of items in the directory: 2279
Number of items in the directory: 2314


# Optional resizing step -- We can resize images and labels to match the correct input size for the model (if the chosen model does not have an inbuilt image_resizer, and if this hasn't already been done locally)

In [ ]:
import cv2
import os

def resize_images(input_folder, output_folder, new_width, new_height):
    """Iterates through all images in a given input folder (input_folder) and resizes them to a specified size.
    run annotation_converter.py afterwards to convert any existing/outstanding xml files to the proper size.
    """
    if not os.path.exists(output_folder):
      os.makedirs(output_folder)


    for filename in os.listdir(input_folder):
      if filename.endswith('JPG'):
        input_path = os.path.join(input_folder, filename)
        image = cv2.imread(input_path)
        if image is not None:
          # Resize the image
          resized_image = cv2.resize(image, (new_width, new_height))

          # Save the resized image to the output folder
          output_path = os.path.join(output_folder, filename)
          cv2.imwrite(output_path, resized_image)

In [ ]:
#uncheck this if you want to resize images in the cloud
resize_images('Tensorflow/workspace/images/collectedimages', 'Tensorflow/workspace/images/resizedcollectedimages', 640, 640)

In [ ]:
import cv2
import os
import xml.etree.ElementTree as ET

def resize_xml_annotations(xml_folder, output_folder, new_size):
    """ Iterates through all xml_files in a given input folder (xml_folder variable)
    and resizes them to map to the same location on a resized image.
    """
    new_width, new_height = new_size

    for xml_file in os.listdir(xml_folder):
        if xml_file.endswith('.xml'):
            xml_path = os.path.join(xml_folder, xml_file)
            tree = ET.parse(xml_path)
            root = tree.getroot()

            image_size = root.find('size')
            image_width = int(image_size.find('width').text)
            image_height = int(image_size.find('height').text)

            width_ratio = new_width / image_width
            height_ratio = new_height / image_height

            for object in root.findall('object'):
                bndbox = object.find('bndbox')

                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)

                xmin = int(xmin * width_ratio)
                ymin = int(ymin * height_ratio)
                xmax = int(xmax * width_ratio)
                ymax = int(ymax * height_ratio)

                bndbox.find('xmin').text = str(xmin)
                bndbox.find('ymin').text = str(ymin)
                bndbox.find('xmax').text = str(xmax)
                bndbox.find('ymax').text = str(ymax)

            # Update the image size in the XML file
            image_size.find('width').text = str(new_width)
            image_size.find('height').text = str(new_height)

            # Save the updated XML file to the output folder
            output_path = os.path.join(output_folder, xml_file)
            tree.write(output_path)

In [ ]:
#uncheck if you want to resize xml labels in the cloud
resize_xml_annotations('Tensorflow/workspace/images/collectedimages', 'Tensorflow/workspace/images/resizedcollectedimages', (640,640))

#2. Train/Test Split

In [ ]:
import os
import random
from sklearn.model_selection import train_test_split
from shutil import copyfile

In [ ]:

# Set the paths to your image and XML directories #resized
image_directory ='Tensorflow/workspace/images/resizedcollectedimages'
xml_directory ='Tensorflow/workspace/images/resizedcollectedimages'

# Create the train and test directories if they don't exist
train_directory = 'Tensorflow/workspace/images/train'
test_directory = 'Tensorflow/workspace/images/test'
os.makedirs(train_directory, exist_ok=True)
os.makedirs(test_directory, exist_ok=True)

# List all image files in the image directory
image_files = [file for file in os.listdir(image_directory) if file.endswith('.JPG')]

# Set the random seed for reproducibility
random_seed = 42

print(image_files)

# Perform the train/test split
train_files, test_files = train_test_split(image_files, test_size=0.2, random_state=random_seed)


# Move the selected images and their corresponding XML files to the train folder
for file in train_files:
    image_src = os.path.join(image_directory, file)
    #print(image_src)
    xml_src = os.path.join(xml_directory, file.replace('.JPG', '.xml'))
    #print(xml_src)
    image_dest = os.path.join(train_directory, file)
    #print(image_dest)
    xml_dest = os.path.join(train_directory, file.replace('.JPG', '.xml'))
    #print(xml_dest)
    if os.path.exists(xml_src):
      copyfile(image_src, image_dest)
      copyfile(xml_src, xml_dest)

# Move the remaining images and their corresponding XML files to the test folder
for file in test_files:
    image_src = os.path.join(image_directory, file)
    xml_src = os.path.join(xml_directory, file.replace('.JPG', '.xml'))
    image_dest = os.path.join(test_directory, file)
    xml_dest = os.path.join(test_directory, file.replace('.JPG', '.xml'))
    if os.path.exists(xml_src):
      copyfile(image_src, image_dest)
      copyfile(xml_src, xml_dest)


#examine train/test split by unchecking below
print("size of training set",len(train_files))
print("size of testing set",len(test_files))

#Number of images being used in this dataset
print("Overall Number of Images Input into Model: " + str(len(train_files) + len(test_files)))

['02a31e97-AllSkyImage007074172.JPG', 'b753343b-AllSkyImage007170885.JPG', '28831f65-AllSkyImage007073791.JPG', 'b49c3c98-AllSkyImage007170667.JPG', '8264d923-AllSkyImage007170710.JPG', 'c388a1b0-AllSkyImage007075017.JPG', '5bcb6411-AllSkyImage007056727.JPG', 'f8c9665e-AllSkyImage007055279.JPG', 'f76fe432-AllSkyImage007074982.JPG', 'f08f4bbc-AllSkyImage007040503.JPG', '85def637-AllSkyImage007171217.JPG', 'd5306168-AllSkyImage007071349.JPG', '8d642dfd-AllSkyImage007056591.JPG', '2584746d-AllSkyImage007071119.JPG', '04f8c19a-AllSkyImage007072267.JPG', 'e68333ee-AllSkyImage007075020.JPG', '682e1e58-AllSkyImage007040289.JPG', 'ce617c2b-AllSkyImage007071322.JPG', '599c00c2-AllSkyImage007072301.JPG', '248cf569-AllSkyImage007038024.JPG', '6ee3b4ab-AllSkyImage007073779.JPG', '44173d14-AllSkyImage007073730.JPG', '95c86f3a-AllSkyImage007056741.JPG', 'dd0263b3-AllSkyImage007071302.JPG', '1853ac16-AllSkyImage007037670.JPG', '7bbbefe4-AllSkyImage007073725.JPG', '2df1604a-AllSkyImage007040576.JPG', 

#3. Download Tensorflow Object Detection API and Models

In [ ]:
#This downloads the object detection API
if not os.path.exists(os.path.join(folderpaths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {folderpaths['APIMODEL_PATH']}

Cloning into 'Tensorflow/models'...
remote: Enumerating objects: 89900, done.
remote: Counting objects: 100% (3608/3608), done.
remote: Compressing objects: 100% (1935/1935), done.
remote: Total 89900 (delta 1907), reused 3278 (delta 1635), pack-reused 86292
Receiving objects: 100% (89900/89900), 606.57 MiB | 24.23 MiB/s, done.
Resolving deltas: 100% (63818/63818), done.


In [ ]:
#Converts .protoc files into readable .py format
!apt-get install protobuf-compiler
!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
protobuf-compiler is already the newest version (3.12.4-1ubuntu7.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
Processing /content/Tensorflow/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#This seems to be the necessary version of tensorflow (doesn't seem to work with tensorflow==2.14.0)
!pip install tensorflow==2.13.0

In [ ]:
VERIFICATION_SCRIPT = os.path.join(folderpaths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# verify if the installation has worked
!python {VERIFICATION_SCRIPT}

In [ ]:
import object_detection

In [ ]:
#This downloads the actual model you plan to use from TF2 model zoo and moves it
!wget {PRETRAINED_MODEL_URL}
!mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {folderpaths['PRETRAINED_MODEL_PATH']}
!cd {folderpaths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

# 4. Create Label Mapping

In [ ]:
#The machine will class objects as numbers, so we introduce label mapping to map those numbers back to recognisable names of our distinct object groups
labels = [{'name':'Aircraft', 'id':1},{'name':'Non-Aircraft', 'id':2}]
with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

#5. Create TF records files

In [ ]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {folderpaths['SCRIPTS_PATH']}

In [ ]:
#This code converts our xml annotations into a machine readable TFRecord file format
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(folderpaths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(folderpaths['ANNOTATION_PATH'], 'train.record')}
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(folderpaths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(folderpaths['ANNOTATION_PATH'], 'test.record')}

# 6. Copy Model Configuration File to Training Folder

In [ ]:
!cp {os.path.join(folderpaths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(folderpaths['CHECKPOINT_PATH'])}

# 7. Update Config For Transfer Learning

This section is where we adjust hyperparameters of our model either to configure the model to simply work on our input data, or to optimise it for best performance.

In [ ]:

import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [ ]:
pipeline_config.train_config.batch_size = 16
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(folderpaths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(folderpaths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(folderpaths['ANNOTATION_PATH'], 'test.record')]

In [ ]:
#change {model_name} pipeline_config.model.{model_name}.num_classes to the name listed in the general config file
pipeline_config.model.ssd.num_classes = len(labels)


In [ ]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:
    f.write(config_text)

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [ ]:
config

#8. Train the model

We specify the number of training steps we want to have the model learn for

In [ ]:
TRAINING_SCRIPT = os.path.join(folderpaths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=7500".format(TRAINING_SCRIPT, folderpaths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [ ]:
print(command)

In [ ]:
!{command}

#9. Evaluate the Model

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, folderpaths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], folderpaths['CHECKPOINT_PATH'])

In [ ]:
print(command)

In [ ]:
#exit after all printed out - runs perpetually
!{command}

## Dashboard to visualise Training and Testing results

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=/content/Tensorflow/workspace/models/{CUSTOM_MODEL_NAME}/train


In [ ]:
%tensorboard --logdir=/content/Tensorflow/workspace/models/{CUSTOM_MODEL_NAME}/eval --port 6008
#/content/Tensorflow/workspace/models/inception_resnet_v2_640x640/eval

In [ ]:
{CUSTOM_MODEL_NAME}

#10. Load Train Model From Checkpoint


In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
# Load pipeline config and build a detection model - we can replace files['PIPELINE_CONFIG'] and folderpaths['CHECKPOINT_PATH'] and 'ckpt-3'

checkpoint_number = 6  # Replace with the actual checkpoint number you want to restore

checkpoint_path = os.path.join(folderpaths['CHECKPOINT_PATH'], f'ckpt-{checkpoint_number}')

configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(checkpoint_path).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 11. Detect from an Image - Using the model we just loaded

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
# This cell specifies the folder with ML predictions/results that you want to visualise - and collects the filenames of the images within.
import os
import random
IMAGE_PATHS = []
# Directory path
directory_path = "Tensorflow/workspace/images/test"  # Replace with your directory path

# List all files in the directory that end with ".jpg"
jpg_files = [f for f in os.listdir(directory_path) if f.endswith(".JPG")]

# Get the full file paths
file_paths = [os.path.join(directory_path, file) for file in jpg_files]

# Print the selected file paths
for file_path in file_paths:
    IMAGE_PATHS.append(file_path)


In [ ]:
import os
import random
import cv2
import numpy as np
import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils

# Create a directory to store annotated images
output_dir = "machine_annotated_images"
os.makedirs(output_dir, exist_ok=True)

# Counter for image file naming
image_counter = 0

# List to store annotated frames
annotated_frames = []

# Loop through the image paths
for IMAGE_PATH in IMAGE_PATHS:
    img = cv2.imread(IMAGE_PATH)
    image_np = np.array(img)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    plt.xticks([])
    plt.yticks([])

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'] + label_id_offset,
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=5,
        min_score_thresh=0.3,  # Edit
        agnostic_mode=False
    )

    # Append the annotated frame to the list
    annotated_frames.append(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))





In [ ]:
# Create a directory to store annotated images
output_dir = "machine_annotated_images"
os.makedirs(output_dir, exist_ok=True)

for index, annotated_frame in enumerate(annotated_frames):
    # Save the machine annotated frame as a JPG file
    output_image_path = os.path.join(output_dir, jpg_files[index])
    cv2.imwrite(output_image_path, cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR))

#  Store outputted data/configs/records in google drive or elsewhere


In [ ]:
#Adjust below version number to represent that this is the nth time you have built a trail detection model with this model's architecture type (specified in {CUSTOM_MODEL_NAME})
version = 3

# Convert CUSTOM_MODEL_NAME to lowercase
custom_model_name = CUSTOM_MODEL_NAME.lower()

# Replace {len(train_files) + len(test_files)} with the actual value
total_images = len(train_files) + len(test_files)  # replace with your actual values


# Set the source and destination folders
source_folder = f'Tensorflow/workspace/models/{custom_model_name}'
destination_folder = f'/content/drive/MyDrive/aircraftimage/output/{custom_model_name}_version_{version}_{total_images}_images'

In [ ]:
output_video_path = f"output_video_{custom_model_name}_version_{version}.mp4"

fps = 1

image_files = [os.path.join(output_dir, jpg) for jpg in os.listdir(output_dir)]

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (annotated_frames[0].shape[1], annotated_frames[0].shape[0]))

for image_file in image_files:
    img = cv2.imread(image_file)
    video_writer.write(img)

video_writer.release()

shutil.move(output_video_path,source_folder)

In [ ]:
shutil.move('/content/machine_annotated_images',source_folder)

In [ ]:
import os
import shutil

# Create the destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Copy individual files and folders from the source folder to the destination folder
for item in os.listdir(source_folder):
    src_item = os.path.join(source_folder, item)
    dest_item = os.path.join(destination_folder, item)

    if os.path.isdir(src_item):
        shutil.copytree(src_item, dest_item)
        print(f'Moved directory {src_item} to {dest_item}')
    else:
        shutil.copy(src_item, dest_item)
        print(f'Moved file {src_item} to {dest_item}')


In [ ]:
# Unmount Google Drive
#drive.flush_and_unmount()

In [ ]:


#shutil.copytree('machine_annotated_images_30%', destination_folder + '/machine_annotated_images_30%')
